## Blitz


In [0]:
from google.colab import drive
drive.mount('/content/drive',force_remount = True)

In [0]:
!ls

In [0]:
import time
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import os
from torchvision import models,transforms,datasets
import os

In [0]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms. ToTensor(),
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        ]),
}

data_dir = 'drive/My Drive/IndIan_Food_DatasetM'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=128,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else print("Enable GPU"))


In [0]:
pretrained = models.vgg16_bn(pretrained=True)


In [0]:
for param in pretrained.parameters():
    pretrained.requires_grad_ = False

## Include in notes

In [8]:
for name, child in pretrained.named_children():
    print(name)

features
avgpool
classifier


In [9]:
for n,c in pretrained.classifier.named_children():
    print(n)

0
1
2
3
4
5
6


## Include

In [10]:
pretrained.fc.out_features #features means (hidden) units

AttributeError: ignored

In [0]:
class TransferLearn(nn.Module):
    def __init__(self):
        super(TransferLearn,self).__init__()
        self.pretrained = models.resnet34(pretrained=True)
        filters = self.pretrained.fc.in_features
        for param in self.pretrained.parameters():
          self.pretrained.requires_grad_ = False    #=INCLUDE=#Testing if we could send the parameters from forward() Hence not setting up the "in/out_features"#
        self.pretrained.fc = nn.Linear(filters,128)                             ###ANSWER IS NO WE CAN'T!##########################
        self.maxpool = nn.MaxPool2d(2)##as per keras' GLobal pooling layer
        self.dropout = nn.Dropout(0.2)
        self.out = nn.Linear(128,6)
    def forward(self,imb):
        x = self.pretrained(imb)
        #x = self.maxpool(x)
        x = self.dropout(x)
        preds = self.out(x)
        return preds

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else print("Enable GPU"))


In [0]:
model = TransferLearn()
model = model.to(device)

In [0]:
if torch.cuda.is_available():
    model.cuda()

In [0]:
criterion = nn.CrossEntropyLoss()


In [0]:
optimizer = optim.SGD(model.parameters(),lr = 0.001,momentum = 0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer,step_size = 7, gamma = 0.1)


In [0]:
def train(model,optimizer,criterion,epochs = 25):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(epochs):
        print("Epoch {}/{}".format(epoch+1,epochs))
        print('-'*10)
        
        for phase in ['train','test']:
            if phase == 'train':
                exp_lr_scheduler.step()
                model.train()
            else:
                model.eval()
            current_loss = 0.0
            current_corrects = 0
            
            print('Iterating through {} data...'.format(phase))
            
            for images,labels in dataloaders[phase]:
#                 print(type(images))
#                 print(len(images))
                images = images.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(images)
                    _,preds = torch.max(outputs,1)
                    loss = criterion(outputs,labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                
                current_loss += loss.item()*images.size(0)
                current_corrects += torch.sum(preds == labels.data)
            
            epoch_loss = current_loss / dataset_sizes[phase]
            epoch_acc = current_corrects.double() /dataset_sizes[phase]
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase,epoch_loss,epoch_acc))
            
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
            print()
            
    time_since = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_since // 60, time_since % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # Now we'll load in the best model weights and return it
    model.load_state_dict(best_model_wts)
    return model        
                

In [12]:
trained_model = train(model,optimizer,criterion)

Epoch 1/25
----------
Iterating through train data...


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


train Loss: 1.6927 Acc: 0.3109

Iterating through test data...
test Loss: 1.2637 Acc: 0.7145

Epoch 2/25
----------
Iterating through train data...
train Loss: 1.1332 Acc: 0.6956

Iterating through test data...
test Loss: 0.5671 Acc: 0.9276

Epoch 3/25
----------
Iterating through train data...
train Loss: 0.7414 Acc: 0.8088

Iterating through test data...
test Loss: 0.2837 Acc: 0.9631

Epoch 4/25
----------
Iterating through train data...
train Loss: 0.5369 Acc: 0.8489

Iterating through test data...
test Loss: 0.1873 Acc: 0.9713

Epoch 5/25
----------
Iterating through train data...
train Loss: 0.4361 Acc: 0.8732

Iterating through test data...
test Loss: 0.1244 Acc: 0.9809

Epoch 6/25
----------
Iterating through train data...
train Loss: 0.3758 Acc: 0.8874

Iterating through test data...
test Loss: 0.1201 Acc: 0.9781

Epoch 7/25
----------
Iterating through train data...
train Loss: 0.3559 Acc: 0.8886

Iterating through test data...
test Loss: 0.0933 Acc: 0.9822

Epoch 8/25
-------

In [23]:
trained_model = train(model,optimizer,criterion)

Epoch 1/2
----------
Iterating through train data...
train Loss: 1.9508 Acc: 0.1798

Iterating through test data...
test Loss: 1.9468 Acc: 0.2281

Epoch 2/2
----------
Iterating through train data...
train Loss: 1.9421 Acc: 0.1869

Iterating through test data...
test Loss: 1.9504 Acc: 0.2022

Training complete in 0m 27s
Best val Acc: 0.228142


In [22]:
trained_model = train(model,optimizer,criterion)

Epoch 1/2
----------
Iterating through train data...
train Loss: 1.9316 Acc: 0.1934

Iterating through test data...
test Loss: 1.9487 Acc: 0.2322

Epoch 2/2
----------
Iterating through train data...
train Loss: 1.9436 Acc: 0.1866

Iterating through test data...
test Loss: 1.9490 Acc: 0.1981

Training complete in 0m 27s
Best val Acc: 0.232240


In [1]:
!nvidia-smi

Sun Dec 29 11:17:08 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [24]:
trained_model = train(model,optimizer,criterion)

Epoch 1/2
----------
Iterating through train data...
train Loss: 1.9423 Acc: 0.1845

Iterating through test data...
test Loss: 1.9410 Acc: 0.2336

Epoch 2/2
----------
Iterating through train data...
train Loss: 1.9400 Acc: 0.1786

Iterating through test data...
test Loss: 1.9516 Acc: 0.1995

Training complete in 0m 27s
Best val Acc: 0.233607


In [25]:
trained_model = train(model,optimizer,criterion)

Epoch 1/2
----------
Iterating through train data...
train Loss: 1.9482 Acc: 0.1909

Iterating through test data...
test Loss: 1.9450 Acc: 0.2309

Epoch 2/2
----------
Iterating through train data...
train Loss: 1.9418 Acc: 0.1965

Iterating through test data...
test Loss: 1.9512 Acc: 0.1995

Training complete in 0m 27s
Best val Acc: 0.230874


In [26]:
trained_model = train(model,optimizer,criterion)

Epoch 1/2
----------
Iterating through train data...
train Loss: 1.9513 Acc: 0.1854

Iterating through test data...
test Loss: 1.9470 Acc: 0.2281

Epoch 2/2
----------
Iterating through train data...
train Loss: 1.9426 Acc: 0.1851

Iterating through test data...
test Loss: 1.9539 Acc: 0.1967

Training complete in 0m 27s
Best val Acc: 0.228142


In [29]:
trained_model = train(model,optimizer,criterion)

Epoch 1/2
----------
Iterating through train data...
train Loss: 1.9408 Acc: 0.1872

Iterating through test data...
test Loss: 1.9463 Acc: 0.2309

Epoch 2/2
----------
Iterating through train data...
train Loss: 1.9445 Acc: 0.1903

Iterating through test data...
test Loss: 1.9519 Acc: 0.2022

Training complete in 0m 28s
Best val Acc: 0.230874


In [30]:
trained_model = train(model,optimizer,criterion)

Epoch 1/2
----------
Iterating through train data...
train Loss: 1.9390 Acc: 0.1851

Iterating through test data...
test Loss: 1.9471 Acc: 0.2322

Epoch 2/2
----------
Iterating through train data...
train Loss: 1.9416 Acc: 0.1956

Iterating through test data...
test Loss: 1.9523 Acc: 0.2008

Training complete in 0m 28s
Best val Acc: 0.232240


In [31]:
trained_model = train(model,optimizer,criterion)

Epoch 1/2
----------
Iterating through train data...
train Loss: 1.9489 Acc: 0.1764

Iterating through test data...
test Loss: 1.9476 Acc: 0.2336

Epoch 2/2
----------
Iterating through train data...
train Loss: 1.9391 Acc: 0.1872

Iterating through test data...
test Loss: 1.9520 Acc: 0.2036

Training complete in 0m 27s
Best val Acc: 0.233607


In [32]:
trained_model = train(model,optimizer,criterion)

Epoch 1/2
----------
Iterating through train data...
train Loss: 1.9578 Acc: 0.1764

Iterating through test data...
test Loss: 1.9472 Acc: 0.2309

Epoch 2/2
----------
Iterating through train data...
train Loss: 1.9472 Acc: 0.1832

Iterating through test data...
test Loss: 1.9472 Acc: 0.2008

Training complete in 0m 28s
Best val Acc: 0.230874


In [33]:
trained_model = train(model,optimizer,criterion,10)

Epoch 1/10
----------
Iterating through train data...
train Loss: 1.9465 Acc: 0.1835

Iterating through test data...
test Loss: 1.9481 Acc: 0.2309

Epoch 2/10
----------
Iterating through train data...
train Loss: 1.9445 Acc: 0.1854

Iterating through test data...
test Loss: 1.9496 Acc: 0.1995

Epoch 3/10
----------
Iterating through train data...
train Loss: 1.9531 Acc: 0.1820

Iterating through test data...
test Loss: 1.9531 Acc: 0.1995

Epoch 4/10
----------
Iterating through train data...
train Loss: 1.9434 Acc: 0.1919

Iterating through test data...
test Loss: 1.9552 Acc: 0.1967

Epoch 5/10
----------
Iterating through train data...
train Loss: 1.9438 Acc: 0.1888

Iterating through test data...
test Loss: 1.9556 Acc: 0.1940

Epoch 6/10
----------
Iterating through train data...
train Loss: 1.9415 Acc: 0.1811

Iterating through test data...
test Loss: 1.9533 Acc: 0.1954

Epoch 7/10
----------
Iterating through train data...
train Loss: 1.9543 Acc: 0.1835

Iterating through test dat

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
plt.imshow(img_datasets[342][0])

In [0]:
img_datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'))

In [0]:
?datasets.ImageFolder()

In [0]:
!cat ../usr/local/lib/python3.6/dist-packages/torchvision/datasets/folder.py